<a href="https://colab.research.google.com/github/kaitz2981-ux/TTNT/blob/main/Tuan3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TTNT-Tuan03


In [ ]:
import cv2
import numpy
import copy
import math
import random

X = "X"
O = "O"
EMPTY = None
user = None # Giá trị này có thể được đặt bởi người dùng sau này (X hoặc O)
ai = None   # Giá trị này có thể được đặt bởi người dùng sau này (X hoặc O)

def initial_state():
    """
    Trả về trạng thái ban đầu của bàn cờ.
    """
    return [[EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY]]

def player(board):
    """
    Trả về người chơi có lượt đi tiếp theo trên bàn cờ.
    """
    count = 0
    for row in board:
        for cell in row:
            if cell is not EMPTY: # Kiểm tra các ô không trống
                count += 1
    if count % 2 != 0: # Nếu số lượng nước đi lẻ, là lượt của AI
        return ai
    return user # Ngược lại, là lượt của người dùng

def actions(board):
    """
    Trả về tập hợp tất cả các hành động có thể (i, j) có sẵn trên bàn cờ.
    """
    res = set()
    board_len = len(board)
    for i in range(board_len):
        for j in range(board_len):
            if board[i][j] == EMPTY:
                res.add((i, j))
    return res

def result(board, action):
    """
    Trả về bàn cờ sau khi thực hiện nước đi (i, j).
    """
    curr_player = player(board)
    result_board = copy.deepcopy(board)
    (i, j) = action
    if result_board[i][j] != EMPTY:
        raise ValueError("Hành động không hợp lệ: ô đã bị chiếm") # Thêm xử lý lỗi
    result_board[i][j] = curr_player
    return result_board

def get_horizontal_winner(board):
    # Kiểm tra theo chiều ngang
    board_len = len(board)
    for i in range(board_len):
        if board[i][0] != EMPTY and board[i][0] == board[i][1] and board[i][1] == board[i][2]:
            return board[i][0]
    return None

def get_vertical_winner(board):
    # Kiểm tra theo chiều dọc
    board_len = len(board)
    for i in range(board_len):
        if board[0][i] != EMPTY and board[0][i] == board[1][i] and board[1][i] == board[2][i]:
            return board[0][i]
    return None

def get_diagonal_winner(board):
    # Kiểm tra theo đường chéo
    # Từ trên cùng bên trái xuống dưới cùng bên phải
    if board[0][0] != EMPTY and board[0][0] == board[1][1] and board[1][1] == board[2][2]:
        return board[0][0]
    # Từ trên cùng bên phải xuống dưới cùng bên trái
    if board[0][2] != EMPTY and board[0][2] == board[1][1] and board[1][1] == board[2][0]:
        return board[0][2]
    return None

def winner(board):
    winner_val = get_horizontal_winner(board) or get_vertical_winner(board) or get_diagonal_winner(board)
    return winner_val

def terminal(board):
    if winner(board) is not None:
        return True
    for row in board:
        for cell in row:
            if cell == EMPTY:
                return False
    return True # Nếu không có người thắng và không còn ô trống, đó là trạng thái hòa/kết thúc

def utility(board):
    winner_val = winner(board)
    if winner_val == X:
        return 1
    elif winner_val == O:
        return -1
    return 0

def maxValue(state):
    if terminal(state):
        return utility(state)
    v = -math.inf
    for action in actions(state):
        v = max(v, minValue(result(state, action)))
    return v

def minValue(state):
    if terminal(state):
        return utility(state)
    v = math.inf
    for action in actions(state):
        v = min(v, maxValue(result(state, action)))
    return v

def minimax(board):
    current_player = player(board)
    if current_player == X: # AI (người chơi tối đa hóa)
        best_score = -math.inf
        best_move = None
        for action in actions(board):
            score = minValue(result(board, action))
            if score > best_score:
                best_score = score
                best_move = action
        return best_move
    elif current_player == O: # AI (người chơi tối thiểu hóa, nếu AI là O)
        best_score = math.inf
        best_move = None
        for action in actions(board):
            score = maxValue(result(board, action))
            if score < best_score:
                best_score = score
                best_move = action
        return best_move
    return None # Không nên xảy ra nếu player() luôn trả về X hoặc O cho trạng thái không kết thúc

def print_board(board):
    for i in range(3):
        print(" | ".join([str(cell if cell is not EMPTY else " ") for cell in board[i]]))
        if i < 2:
            print("-----------")

if __name__ == "__main__":
    board = initial_state()
    user = X # Giả sử người dùng chơi X
    ai = O # Giả sử AI chơi O
    ai_turn = False # Bắt đầu với lượt của người dùng (X)
    print("Trò chơi Tic-Tac-Toe đã bắt đầu!")
    print_board(board)

    # Vòng lặp trò chơi
    while not terminal(board):
        if ai_turn:
            print("Lượt của AI...")
            move = minimax(board)
            board = result(board, move)
            ai_turn = False
        else:
            print("Lượt của bạn. Nhập nước đi theo định dạng 'hàng,cột' (ví dụ: 1,2)")
            possible_moves = actions(board)
            move = None
            while move not in possible_moves:
                try:
                    user_input = input("Nhập nước đi của bạn: ")
                    # Chuyển đổi từ 1-indexed sang 0-indexed
                    row, col = map(int, user_input.split(','))
                    if row < 1 or col < 1 or row > 3 or col > 3:
                        print("Hàng và cột phải là số từ 1 đến 3. Vui lòng thử lại.")
                        continue

                    # Adjust for 0-indexed board
                    adjusted_row = row - 1
                    adjusted_col = col - 1

                    if (adjusted_row, adjusted_col) in possible_moves:
                        move = (adjusted_row, adjusted_col)
                    else:
                        print("Nước đi không hợp lệ hoặc ô đã bị chiếm. Vui lòng thử lại.")
                except ValueError:
                    print("Định dạng nhập không hợp lệ. Vui lòng nhập 'hàng,cột' (ví dụ: 1,2).")
                except IndexError:
                    print("Định dạng nhập không hợp lệ. Vui lòng nhập 'hàng,cột' (ví dụ: 1,2).")

            board = result(board, move)
            ai_turn = True
        print_board(board)

    final_winner = winner(board)
    if final_winner:
        print(f"Người thắng: {final_winner}")
    else:
        print("Hòa!")

Trò chơi Tic-Tac-Toe đã bắt đầu!
  |   |  
-----------
  |   |  
-----------
  |   |  
Lượt của bạn. Nhập nước đi theo định dạng 'hàng,cột' (ví dụ: 1,2)
Nhập nước đi của bạn: 1, 1
X |   |  
-----------
  |   |  
-----------
  |   |  
Lượt của AI...
X |   |  
-----------
  | O |  
-----------
  |   |  
Lượt của bạn. Nhập nước đi theo định dạng 'hàng,cột' (ví dụ: 1,2)
Nhập nước đi của bạn: 1, 3
X |   | X
-----------
  | O |  
-----------
  |   |  
Lượt của AI...
X | O | X
-----------
  | O |  
-----------
  |   |  
Lượt của bạn. Nhập nước đi theo định dạng 'hàng,cột' (ví dụ: 1,2)
Nhập nước đi của bạn: 3, 2
X | O | X
-----------
  | O |  
-----------
  | X |  
Lượt của AI...
X | O | X
-----------
O | O |  
-----------
  | X |  
Lượt của bạn. Nhập nước đi theo định dạng 'hàng,cột' (ví dụ: 1,2)
Nhập nước đi của bạn: 2, 3
X | O | X
-----------
O | O | X
-----------
  | X |  
Lượt của AI...
X | O | X
-----------
O | O | X
-----------
  | X | O
Lượt của bạn. Nhập nước đi theo định dạng 'hàng,c

# Task
Refactor the Tic-Tac-Toe game to support variable board sizes (NxN) and a configurable K-in-a-row win condition, then test the game with different board sizes (e.g., 3x3, 4x4, 5x5) and confirm that both the user and AI can play and achieve victory or a draw under the new conditions.

## Giới thiệu biến kích thước bàn cờ và điều kiện thắng

### Subtask:
Thêm các biến toàn cục `BOARD_SIZE` và `K_TO_WIN` để định cấu hình kích thước bàn cờ (NxN) và số lượng biểu tượng liên tiếp cần thiết để thắng (K-in-a-row). Thay đổi `initial_state()` để tạo bàn cờ NxN.


**Reasoning**:
The subtask requires adding two new global variables, `BOARD_SIZE` and `K_TO_WIN`, and modifying the `initial_state()` function to use `BOARD_SIZE`. I will create a `code_block` that incorporates these changes into the relevant part of the existing code.



In [ ]:
import cv2
import numpy
import copy
import math
import random

X = "X"
O = "O"
EMPTY = None
user = None # Giá trị này có thể được đặt bởi người dùng sau này (X hoặc O)
ai = None   # Giá trị này có thể được đặt bởi người dùng sau này (X hoặc O)

# Khai báo biến toàn cục mới
BOARD_SIZE = 3
K_TO_WIN = 3

def initial_state():
    """
    Trả về trạng thái ban đầu của bàn cờ.
    """
    # Sửa đổi để tạo bàn cờ NxN
    return [[EMPTY for _ in range(BOARD_SIZE)] for _ in range(BOARD_SIZE)]

def player(board):
    """
    Trả về người chơi có lượt đi tiếp theo trên bàn cờ.
    """
    count = 0
    for row in board:
        for cell in row:
            if cell is not EMPTY: # Kiểm tra các ô không trống
                count += 1
    if count % 2 != 0: # Nếu số lượng nước đi lẻ, là lượt của AI
        return ai
    return user # Ngược lại, là lượt của người dùng

def actions(board):
    """
    Trả về tập hợp tất cả các hành động có thể (i, j) có sẵn trên bàn cờ.
    """
    res = set()
    board_len = len(board)
    for i in range(board_len):
        for j in range(board_len):
            if board[i][j] == EMPTY:
                res.add((i, j))
    return res

def result(board, action):
    """
    Trả về bàn cờ sau khi thực hiện nước đi (i, j).
    """
    curr_player = player(board)
    result_board = copy.deepcopy(board)
    (i, j) = action
    if result_board[i][j] != EMPTY:
        raise ValueError("Hành động không hợp lệ: ô đã bị chiếm") # Thêm xử lý lỗi
    result_board[i][j] = curr_player
    return result_board

def get_horizontal_winner(board):
    # Kiểm tra theo chiều ngang
    board_len = len(board)
    for i in range(board_len):
        if board[i][0] != EMPTY and board[i][0] == board[i][1] and board[i][1] == board[i][2]:
            return board[i][0]
    return None

def get_vertical_winner(board):
    # Kiểm tra theo chiều dọc
    board_len = len(board)
    for i in range(board_len):
        if board[0][i] != EMPTY and board[0][i] == board[1][i] and board[1][i] == board[2][i]:
            return board[0][i]
    return None

def get_diagonal_winner(board):
    # Kiểm tra theo đường chéo
    # Từ trên cùng bên trái xuống dưới cùng bên phải
    if board[0][0] != EMPTY and board[0][0] == board[1][1] and board[1][1] == board[2][2]:
        return board[0][0]
    # Từ trên cùng bên phải xuống dưới cùng bên trái
    if board[0][2] != EMPTY and board[0][2] == board[1][1] and board[1][1] == board[2][0]:
        return board[0][2]
    return None

def winner(board):
    winner_val = get_horizontal_winner(board) or get_vertical_winner(board) or get_diagonal_winner(board)
    return winner_val

def terminal(board):
    if winner(board) is not None:
        return True
    for row in board:
        for cell in row:
            if cell == EMPTY:
                return False
    return True # Nếu không có người thắng và không còn ô trống, đó là trạng thái hòa/kết thúc

def utility(board):
    winner_val = winner(board)
    if winner_val == X:
        return 1
    elif winner_val == O:
        return -1
    return 0

def maxValue(state):
    if terminal(state):
        return utility(state)
    v = -math.inf
    for action in actions(state):
        v = max(v, minValue(result(state, action)))
    return v

def minValue(state):
    if terminal(state):
        return utility(state)
    v = math.inf
    for action in actions(state):
        v = min(v, maxValue(result(state, action)))
    return v

def minimax(board):
    current_player = player(board)
    if current_player == X: # AI (người chơi tối đa hóa)
        best_score = -math.inf
        best_move = None
        for action in actions(board):
            score = minValue(result(board, action))
            if score > best_score:
                best_score = score
                best_move = action
        return best_move
    elif current_player == O: # AI (người chơi tối thiểu hóa, nếu AI là O)
        best_score = math.inf
        best_move = None
        for action in actions(board):
            score = maxValue(result(board, action))
            if score < best_score:
                best_score = score
                best_move = action
        return best_move
    return None # Không nên xảy ra nếu player() luôn trả về X hoặc O cho trạng thái không kết thúc

def print_board(board):
    for i in range(3):
        print(" | ".join([str(cell if cell is not EMPTY else " ") for cell in board[i]]))
        if i < 2:
            print("-----------")

if __name__ == "__main__":
    board = initial_state()
    user = X # Giả sử người dùng chơi X
    ai = O # Giả sử AI chơi O
    ai_turn = False # Bắt đầu với lượt của người dùng (X)
    print("Trò chơi Tic-Tac-Toe đã bắt đầu!")
    print_board(board)

    # Vòng lặp trò chơi
    while not terminal(board):
        if ai_turn:
            print("Lượt của AI...")
            move = minimax(board)
            board = result(board, move)
            ai_turn = False
        else:
            print("Lượt của bạn. Nhập nước đi theo định dạng 'hàng,cột' (ví dụ: 1,2)")
            possible_moves = actions(board)
            move = None
            while move not in possible_moves:
                try:
                    user_input = input("Nhập nước đi của bạn: ")
                    # Chuyển đổi từ 1-indexed sang 0-indexed
                    row, col = map(int, user_input.split(','))
                    if row < 1 or col < 1 or row > 3 or col > 3:
                        print("Hàng và cột phải là số từ 1 đến 3. Vui lòng thử lại.")
                        continue

                    # Adjust for 0-indexed board
                    adjusted_row = row - 1
                    adjusted_col = col - 1

                    if (adjusted_row, adjusted_col) in possible_moves:
                        move = (adjusted_row, adjusted_col)
                    else:
                        print("Nước đi không hợp lệ hoặc ô đã bị chiếm. Vui lòng thử lại.")
                except ValueError:
                    print("Định dạng nhập không hợp lệ. Vui lòng nhập 'hàng,cột' (ví dụ: 1,2).")
                except IndexError:
                    print("Định dạng nhập không hợp lệ. Vui lòng nhập 'hàng,cột' (ví dụ: 1,2).")

            board = result(board, move)
            ai_turn = True
        print_board(board)

    final_winner = winner(board)
    if final_winner:
        print(f"Người thắng: {final_winner}")
    else:
        print("Hòa!")

Trò chơi Tic-Tac-Toe đã bắt đầu!
  |   |  
-----------
  |   |  
-----------
  |   |  
Lượt của bạn. Nhập nước đi theo định dạng 'hàng,cột' (ví dụ: 1,2)
Nhập nước đi của bạn: 1,2
  | X |  
-----------
  |   |  
-----------
  |   |  
Lượt của AI...
  | X |  
-----------
  |   |  
-----------
  | O |  
Lượt của bạn. Nhập nước đi theo định dạng 'hàng,cột' (ví dụ: 1,2)
Nhập nước đi của bạn: 2,2
  | X |  
-----------
  | X |  
-----------
  | O |  
Lượt của AI...
O | X |  
-----------
  | X |  
-----------
  | O |  
Lượt của bạn. Nhập nước đi theo định dạng 'hàng,cột' (ví dụ: 1,2)
Nhập nước đi của bạn: 3,1
O | X |  
-----------
  | X |  
-----------
X | O |  
Lượt của AI...
O | X | O
-----------
  | X |  
-----------
X | O |  
Lượt của bạn. Nhập nước đi theo định dạng 'hàng,cột' (ví dụ: 1,2)
Nhập nước đi của bạn: 2,1
O | X | O
-----------
X | X |  
-----------
X | O |  
Lượt của AI...
O | X | O
-----------
X | X | O
-----------
X | O |  
Lượt của bạn. Nhập nước đi theo định dạng 'hàng,cột' 

In [ ]:
import math
import copy

# Cài đặt chung
PLAYER_X = 'X' # Người chơi Tối đa hóa (Maximizer)
PLAYER_O = 'O' # Người chơi Tối thiểu hóa (Minimizer)
EMPTY = ' '

class GomokuAI_AlphaBeta:

    def __init__(self, size=5, k_win=4, max_depth=3):
        """
        Khởi tạo bàn cờ NxN và điều kiện thắng K
        :param size: Kích thước bàn cờ (N)
        :param k_win: Số quân liên tiếp để thắng (K)
        :param max_depth: Độ sâu tìm kiếm tối đa (để giới hạn thời gian)
        """
        self.N = size
        self.K_WIN = k_win
        self.MAX_DEPTH = max_depth
        self.board = [[EMPTY for _ in range(self.N)] for _ in range(self.N)]

        # Điểm số Heuristic cho các pattern quan trọng
        self.SCORE_WIN = 100000000 # Điểm thắng tuyệt đối
        self.SCORES = {
            self.K_WIN - 1: 50000,  # 3 quân (nếu K=4) hoặc 4 quân (nếu K=5)
            self.K_WIN - 2: 1000,   # 2 quân (nếu K=4) hoặc 3 quân (nếu K=5)
            self.K_WIN - 3: 50      # 1 quân (nếu K=4) hoặc 2 quân (nếu K=5)
        }

    # --- CÁC HÀM KIỂM TRA TRẠNG THÁI ---

    def _check_win_NxN(self, player):
        """
        Kiểm tra xem người chơi đã thắng K quân liên tiếp chưa.
        """
        directions = [(0, 1), (1, 0), (1, 1), (1, -1)] # Ngang, Dọc, Chéo xuôi, Chéo ngược

        for r in range(self.N):
            for c in range(self.N):
                if self.board[r][c] == player:
                    for dr, dc in directions:
                        count = 1
                        for k in range(1, self.K_WIN):
                            nr, nc = r + k * dr, c + k * dc
                            if 0 <= nr < self.N and 0 <= nc < self.N and self.board[nr][nc] == player:
                                count += 1
                            else:
                                break
                        if count >= self.K_WIN:
                            return True
        return False

    def _is_moves_left(self):
        """
        Kiểm tra còn ô trống để đi hay không.
        """
        for row in self.board:
            if EMPTY in row:
                return True
        return False

    # --- HÀM ĐÁNH GIÁ HEURISTIC (TRỌNG TÂM CỦA AI NXN) ---

    def _evaluate_line(self, line, player):
        """
        Đánh giá giá trị của một đường (ngang, dọc, chéo) dựa trên các pattern.
        """
        score = 0
        opponent = PLAYER_O if player == PLAYER_X else PLAYER_X

        for length, point in self.SCORES.items():
            # Pattern cần tìm, ví dụ: ' X X X ' (K-1 quân trống 2 đầu)
            # Hàm này cần được triển khai chi tiết để tìm các chuỗi bị chặn/mở
            # Trong phiên bản đơn giản này, ta chỉ đếm các chuỗi liên tiếp không bị chặn.

            # Đếm số quân liên tiếp của Player
            for i in range(len(line) - length + 1):
                sub_line = line[i : i + length]

                # Nếu chuỗi có đủ độ dài K-n
                if sub_line.count(player) == length and sub_line.count(EMPTY) == 0:

                    # Kiểm tra 2 đầu xem có bị chặn bởi đối thủ không
                    blocked_start = (i > 0 and line[i - 1] == opponent)
                    blocked_end = (i + length < len(line) and line[i + length] == opponent)

                    if not blocked_start and not blocked_end:
                        score += point # Chuỗi mở 2 đầu (Rất mạnh)
                    elif not blocked_start or not blocked_end:
                        score += point // 10 # Chuỗi bị chặn 1 đầu

        return score

    def heuristic_evaluate(self, ai_player):
        """
        Hàm đánh giá Heuristic chính.
        """
        human_player = PLAYER_O if ai_player == PLAYER_X else PLAYER_X

        # 1. Kiểm tra thắng/thua trực tiếp (điểm tối đa)
        if self._check_win_NxN(ai_player): return self.SCORE_WIN
        if self._check_win_NxN(human_player): return -self.SCORE_WIN

        # 2. Đánh giá trạng thái bàn cờ hiện tại
        score_ai = 0
        score_human = 0
        directions = [(0, 1), (1, 0), (1, 1), (1, -1)]

        # Tạo danh sách tất cả các đường (line) trên bàn cờ
        lines = []
        for r in range(self.N):
            # Hàng ngang
            lines.append(self.board[r])
            # Hàng dọc
            lines.append([self.board[i][r] for i in range(self.N)])

        # Hàng chéo (chỉ các chéo chính và phụ lớn)
        for i in range(-self.N + self.K_WIN, self.N - self.K_WIN + 1):
            # Chéo xuôi (i = c - r)
            lines.append([self.board[r][r + i] for r in range(self.N) if 0 <= r + i < self.N])
            # Chéo ngược (i = r + c)
            lines.append([self.board[r][i - r] for r in range(self.N) if 0 <= i - r < self.N])

        # Đánh giá từng đường
        for line in lines:
            if len(line) >= self.K_WIN:
                score_ai += self._evaluate_line(line, ai_player)
                score_human += self._evaluate_line(line, human_player)

        # Trả về điểm số chênh lệch
        return score_ai - score_human

    # --- THUẬT TOÁN ALPHA-BETA PRUNING ---

    def minimax_alpha_beta(self, board, depth, alpha, beta, is_maximizing_player, ai_player):

        # 1. Kiểm tra điều kiện dừng: Độ sâu tối đa, Hết nước đi, hoặc Trò chơi kết thúc
        if depth == self.MAX_DEPTH or not self._is_moves_left() or \
           self._check_win_NxN(ai_player) or self._check_win_NxN(PLAYER_O if ai_player == PLAYER_X else PLAYER_X):

            # Sử dụng Heuristic để đánh giá trạng thái
            score = self.heuristic_evaluate(ai_player)

            # Giảm điểm theo độ sâu (giá trị hiện tại tốt hơn giá trị tương lai)
            if score > 0: return score - depth
            if score < 0: return score + depth
            return score

        # 2. Người chơi tối đa hóa (AI)
        if is_maximizing_player:
            max_eval = -math.inf

            # Lặp qua tất cả các nước đi khả dĩ
            for r in range(self.N):
                for c in range(self.N):
                    if board[r][c] == EMPTY:

                        # Thử nước đi
                        board[r][c] = ai_player
                        eval = self.minimax_alpha_beta(board, depth + 1, alpha, beta, False, ai_player)
                        board[r][c] = EMPTY # Quay lui (Undo move)

                        max_eval = max(max_eval, eval)
                        alpha = max(alpha, max_eval)

                        # Cắt tỉa Beta
                        if beta <= alpha: return max_eval

            return max_eval

        # 3. Người chơi tối thiểu hóa (Đối thủ)
        else:
            min_eval = math.inf
            human_player = PLAYER_O if ai_player == PLAYER_X else PLAYER_X

            for r in range(self.N):
                for c in range(self.N):
                    if board[r][c] == EMPTY:

                        # Thử nước đi
                        board[r][c] = human_player
                        eval = self.minimax_alpha_beta(board, depth + 1, alpha, beta, True, ai_player)
                        board[r][c] = EMPTY # Quay lui (Undo move)

                        min_eval = min(min_eval, eval)
                        beta = min(beta, min_eval)

                        # Cắt tỉa Alpha
                        if beta <= alpha: return min_eval

            return min_eval

    def find_best_move(self, current_board, ai_player):
        """
        Tìm nước đi tối ưu nhất cho AI.
        """
        temp_board = copy.deepcopy(current_board)
        best_move_value = -math.inf
        best_move = (-1, -1)

        # Chỉ tìm kiếm trong một số ô gần nhất để tối ưu tốc độ (Quan trọng cho NxN)
        # Trong ví dụ này, ta tìm kiếm toàn bộ bàn cờ để đơn giản.

        for r in range(self.N):
            for c in range(self.N):
                if temp_board[r][c] == EMPTY:
                    temp_board[r][c] = ai_player

                    # Gọi Minimax_Alpha_Beta cho đối thủ (False)
                    move_value = self.minimax_alpha_beta(temp_board, 0, -math.inf, math.inf, False, ai_player)

                    temp_board[r][c] = EMPTY # Quay lui

                    if move_value > best_move_value:
                        best_move_value = move_value
                        best_move = (r, c)

        print(f"Giá trị tốt nhất tìm được: {best_move_value}")
        return best_move

# --- VÍ DỤ CHẠY THỬ ---

def print_board(board):
    print("Bàn cờ hiện tại:")
    for row in board:
        print(" | ".join([str(cell if cell is not EMPTY else " ") for cell in row]))
        print("-" * (len(row) * 2 + 1))

if __name__ == "__main__":

    N_SIZE = 5      # Kích thước 5x5
    K_TO_WIN = 4    # 4 quân liên tiếp để thắng

    ai_engine = GomokuAI_AlphaBeta(size=N_SIZE, k_win=K_TO_WIN, max_depth=3)

    # Thiết lập trạng thái bàn cờ ban đầu (Input động)
    # Ví dụ: Tạo ra 3 quân X liên tiếp để AI (O) phải ngăn chặn.
    initial_board = ai_engine.board
    initial_board[2][2] = PLAYER_X
    initial_board[2][3] = PLAYER_X
    initial_board[2][4] = PLAYER_X

    print(f"Chạy AI trên bàn cờ {N_SIZE}x{N_SIZE} (thắng với {K_TO_WIN} quân)")

    # Giả sử AI là người chơi 'O' (Tối thiểu hóa)
    AI_PLAYER = PLAYER_O

    print_board(initial_board)

    # Nếu AI là O, ta phải gọi Minimax để tìm nước đi tối thiểu (Minimizer)
    # Tuy nhiên, hàm find_best_move được thiết kế cho Maximizer (X) nên ta đảo vai trò tạm thời
    # để AI tìm nước đi tốt nhất cho mình.

    best_move = ai_engine.find_best_move(initial_board, AI_PLAYER)

    r, c = best_move

    if r != -1:
        initial_board[r][c] = AI_PLAYER
        print(f"Nước đi tối ưu nhất cho AI ({AI_PLAYER}) là: ({r}, {c})")
        print_board(initial_board)
    else:
        print("Không tìm thấy nước đi tối ưu hoặc bàn cờ đã đầy.")

Chạy AI trên bàn cờ 5x5 (thắng với 4 quân)
Bàn cờ hiện tại:
  |   |   |   |  
-----------
  |   |   |   |  
-----------
  |   | X | X | X
-----------
  |   |   |   |  
-----------
  |   |   |   |  
-----------
Giá trị tốt nhất tìm được: -52397
Nước đi tối ưu nhất cho AI (O) là: (0, 0)
Bàn cờ hiện tại:
O |   |   |   |  
-----------
  |   |   |   |  
-----------
  |   | X | X | X
-----------
  |   |   |   |  
-----------
  |   |   |   |  
-----------


In [ ]:
import math

class ChessAI:

    def __init__(self, max_depth=4):
        """
        Khởi tạo AI Cờ Vua.
        :param max_depth: Độ sâu tìm kiếm tối đa (thường là 4-6 cho cờ vua).
        """
        self.MAX_DEPTH = max_depth
        self.TRANSPOSITION_TABLE = {} # Dùng để lưu trữ các trạng thái đã tính toán

    # =========================================================================
    # HÀM BẮT BUỘC SỐ 1: BỘ TẠO NƯỚC ĐI HỢP LỆ (Move Generator)
    # PHẦN PHỨC TẠP NHẤT - CẦN MÃ HÓA TẤT CẢ LUẬT CỜ
    # =========================================================================
    def generate_legal_moves(self, board):
        """
        Trả về danh sách tất cả các nước đi hợp lệ (legal moves) từ trạng thái hiện tại.
        """
        # (Giả định rằng hàm này đã được triển khai để tạo ra các đối tượng Nước đi)
        return [move1, move2, move3, ...]

    # =========================================================================
    # HÀM BẮT BUỘC SỐ 2: HÀM ĐÁNH GIÁ HEURISTIC (Evaluation Function)
    # PHẦN QUAN TRỌNG NHẤT CHO HIỆU SUẤT AI
    # =========================================================================
    def evaluate_board(self, board, player_to_move):
        """
        Gán một giá trị số cho trạng thái bàn cờ hiện tại.
        Giá trị dương: Lợi thế cho người chơi Tối đa hóa (Ví dụ: Trắng).
        Giá trị âm: Lợi thế cho người chơi Tối thiểu hóa (Ví dụ: Đen).
        """
        # 1. Giá trị Vật chất (Material Value): Tổng điểm quân cờ
        material_score = self._calculate_material(board)

        # 2. Giá trị Vị trí (Positional Value): Dùng Piece-Square Tables
        positional_score = self._calculate_positional(board)

        # 3. Các yếu tố khác: An toàn của Vua, tính di động, v.v.
        dynamic_score = self._calculate_dynamic_factors(board)

        # Tổng hợp điểm số (ví dụ: 80% Material + 20% Positional)
        final_score = material_score + positional_score + dynamic_score

        # Nếu đã ở trạng thái kết thúc (Thắng/Thua/Hòa), trả về điểm cực đại/cực tiểu
        if self._is_checkmate(board):
            return 999999999  # Thắng
        if self._is_stalemate(board):
            return 0  # Hòa

        return final_score

    # =========================================================================
    # THUẬT TOÁN ALPHA-BETA PRUNING CỐT LÕI
    # =========================================================================
    def minimax_alpha_beta(self, board, depth, alpha, beta, is_maximizing_player):

        # 1. ĐIỀU KIỆN DỪNG:
        if depth == 0 or self._is_game_over(board):
            # Khi chạm độ sâu tối đa, dừng lại và sử dụng Heuristic để đánh giá
            return self.evaluate_board(board, is_maximizing_player)

        # 2. TÌM KIẾM CẮT TỈA (TRANSPOSITION TABLE)
        # Nếu trạng thái này đã được tính toán ở độ sâu lớn hơn, sử dụng lại kết quả.
        # if board in self.TRANSPOSITION_TABLE:
        #     return self.TRANSPOSITION_TABLE[board]

        # 3. Người chơi TỐI ĐA HÓA (MAXIMIZER - Ví dụ: Trắng)
        if is_maximizing_player:
            max_eval = -math.inf

            # Lặp qua tất cả nước đi hợp lệ (generate_legal_moves)
            for move in self.generate_legal_moves(board):

                new_board = self._make_move(board, move) # Thực hiện nước đi

                # Gọi đệ quy cho người chơi Tối thiểu hóa (Minimizer)
                eval = self.minimax_alpha_beta(new_board, depth - 1, alpha, beta, False)

                max_eval = max(max_eval, eval)
                alpha = max(alpha, max_eval) # Cập nhật Alpha

                # CẮT TỈA BETA: Nếu Alpha vượt Beta, nhánh này sẽ không bao giờ được chọn
                if beta <= alpha:
                    break

            # self.TRANSPOSITION_TABLE[board] = max_eval # Lưu kết quả vào bảng
            return max_eval

        # 4. Người chơi TỐI THIỂU HÓA (MINIMIZER - Ví dụ: Đen)
        else:
            min_eval = math.inf

            for move in self.generate_legal_moves(board):

                new_board = self._make_move(board, move) # Thực hiện nước đi

                # Gọi đệ quy cho người chơi Tối đa hóa (Maximizer)
                eval = self.minimax_alpha_beta(new_board, depth - 1, alpha, beta, True)

                min_eval = min(min_eval, eval)
                beta = min(beta, min_eval) # Cập nhật Beta

                # CẮT TỈA ALPHA: Nếu Beta thấp hơn Alpha, nhánh này sẽ không bao giờ được chọn
                if beta <= alpha:
                    break

            # self.TRANSPOSITION_TABLE[board] = min_eval # Lưu kết quả vào bảng
            return min_eval

    # =========================================================================
    # HÀM TÌM KIẾM NƯỚC ĐI TỐT NHẤT (ENTRY POINT)
    # =========================================================================
    def find_best_move(self, board):
        best_move_value = -math.inf
        best_move = None

        for move in self.generate_legal_moves(board):
            new_board = self._make_move(board, move)

            # Gọi hàm Minimax_Alpha_Beta
            # Bắt đầu với lượt của đối thủ (Minimizer), nên is_maximizing_player = False
            move_value = self.minimax_alpha_beta(new_board, self.MAX_DEPTH - 1, -math.inf, math.inf, False)

            if move_value > best_move_value:
                best_move_value = move_value
                best_move = move

        return best_move, best_move_value

    # --- Các hàm hỗ trợ (simplified) ---
    def _is_game_over(self, board): return False
    def _make_move(self, board, move): return board
    def _is_checkmate(self, board): return False
    def _is_stalemate(self, board): return False
    def _calculate_material(self, board): return 0
    def _calculate_positional(self, board): return 0
    def _calculate_dynamic_factors(self, board): return 0

In [ ]:
# --- 1. CÀI ĐẶT VÀ KHỞI TẠO Xvfb ---
!apt-get install -y xvfb xserver-xephyr
import os
os.environ['DISPLAY'] = ':99.0'
# Khởi động Xvfb
!Xvfb :99 -screen 0 1400x960x24 > /dev/null 2>&1 &

# Cho phép các ứng dụng kết nối với Xvfb
!xhost +

# Đợi 2 giây để chắc chắn Xvfb đã khởi động
import time
time.sleep(2)
print("Môi trường hiển thị ảo đã sẵn sàng!")

# --- 2. LOGIC GAME VÀ GUI TKINTER ---

from tkinter import Tk, Button, Frame, messagebox, Label
import sys
import math

# [GIỮ NGUYÊN CÁC HÀM LOGIC CỦA BẠN]
# winner, terminal, get_player, actions, result, utility, minimax,
# minValue_simulated, maxValue_simulated

# --- THIẾT LẬP CÁC HẰNG SỐ ---
X = "X"
O = "O"
EMPTY = None
SIZE = 3 # Kích thước 3x3
AI_PLAYER = O # AI luôn là O

class TicTacToeGUI:
    def __init__(self, master):
        self.master = master
        master.title("Tic-Tac-Toe AI (Tkinter)")

        self.board = [[EMPTY] * SIZE for _ in range(SIZE)]
        self.buttons = []
        self.current_player = X # Người chơi (User) đi trước
        self.game_over = False

        self.status_label = Label(master, text=f"Lượt của Người chơi ({self.current_player})", font=('Arial', 16))
        self.status_label.grid(row=0, column=0, columnspan=SIZE)

        self._create_board_buttons()

    def _create_board_buttons(self):
        """Tạo các nút bấm cho bàn cờ 3x3"""
        board_frame = Frame(self.master)
        board_frame.grid(row=1, column=0, columnspan=SIZE)

        for r in range(SIZE):
            row_buttons = []
            for c in range(SIZE):
                btn = Button(board_frame, text=" ", font=('Arial', 40), width=4, height=2,
                             command=lambda r=r, c=c: self._handle_click(r, c))
                btn.grid(row=r, column=c)
                row_buttons.append(btn)
            self.buttons.append(row_buttons)

    def _handle_click(self, r, c):
        """Xử lý sự kiện click chuột của Người dùng"""
        if self.board[r][c] == EMPTY and not self.game_over and self.current_player == X:
            self.board[r][c] = X
            self.buttons[r][c].config(text=X, fg='blue')
            self._update_game_state()

            # Chuyển sang lượt AI nếu game chưa kết thúc
            if not self.game_over:
                self.current_player = O
                self.status_label.config(text=f"Lượt của AI ({self.current_player}). Đang tính toán...")
                # Sử dụng master.after để cho phép GUI cập nhật trước khi AI tính toán
                self.master.after(500, self._ai_move)

    def _ai_move(self):
        """Lượt đi của AI"""

        # 1. GỌI HÀM MINIMAX CỦA BẠN
        # Đảm bảo hàm minimax() của bạn hoạt động với ma trận 3x3
        ai_move = minimax(self.board)

        if ai_move:
            r, c = ai_move
            self.board[r][c] = AI_PLAYER
            self.buttons[r][c].config(text=AI_PLAYER, fg='red')

        self._update_game_state()

        # Chuyển sang lượt Người dùng
        if not self.game_over:
            self.current_player = X
            self.status_label.config(text=f"Lượt của Người chơi ({self.current_player})")

    def _update_game_state(self):
        """Kiểm tra người thắng và cập nhật trạng thái game"""

        # 1. Kiểm tra trạng thái kết thúc (Sử dụng hàm logic của bạn)
        if terminal(self.board):
            self.game_over = True
            w = winner(self.board)

            if w == X:
                messagebox.showinfo("Game Over", "Người chơi (X) Thắng!")
            elif w == O:
                messagebox.showinfo("Game Over", "AI (O) Thắng!")
            else:
                messagebox.showinfo("Game Over", "Hòa!")

            # Đóng cửa sổ sau khi thông báo
            self.master.quit() # Đóng cửa sổ

# --- PHẦN CHẠY CHÍNH ---
if __name__ == "__main__":
    # Việc chạy Tkinter trong cùng một cell sau khi Xvfb khởi động là cách hiệu quả nhất
    try:
        root = Tk()
        game_gui = TicTacToeGUI(root)
        root.mainloop()
    except Exception as e:
        print(f"Lỗi chạy Tkinter: {e}")
        print("Vui lòng đảm bảo các hàm logic đã được định nghĩa và Xvfb đã khởi động.")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
xserver-xephyr is already the newest version (2:21.1.4-2ubuntu1.7~22.04.16).
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.16).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.
/bin/bash: line 1: xhost: command not found
Môi trường hiển thị ảo đã sẵn sàng!
